<a href="https://colab.research.google.com/github/uzn1023/LLM/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RAG


In [1]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

## 準備


In [3]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 \
    langchain-community==0.3.0 GitPython==3.1.43 \
    langchain-chroma==0.1.4 tavily-python==0.5.0 \
    pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 4.8 MB/s eta 0:00:00


In [20]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 読み込むPDFファイルのリスト
pdf_paths = ["AKreport.pdf", "JPindust.pdf"]

# すべてのドキュメントを格納するリスト
all_documents = []

# 各PDFを読み込む
for pdf_path in pdf_paths:
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    all_documents.extend(documents)

# チャンク化（400文字程度）
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
split_docs = text_splitter.split_documents(all_documents)

# 結果の確認
print(f"読み込んだ元のページ数: {len(all_documents)}")
print(f"チャンク数: {len(split_docs)}")
print(f"最初のチャンクの内容:\n{split_docs[0].page_content if split_docs else 'No content found'}")


読み込んだ元のページ数: 245
チャンク数: 860
最初のチャンクの内容:
旭化成レポート2024
Be a Trailblazer


In [21]:
QUESTION = "旭化成のこれまでのDXへの取り組みと現状の問題、今後必要な取り組みをそれぞれ教えて"

## RAGを使わず生成AIモデルに質問する場合

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# プロンプトを定義（文脈なし）
prompt = ChatPromptTemplate.from_template("以下の質問に答えてください。\n\n質問: {question}")

# LLMの設定
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | model | StrOutputParser()

response = chain.invoke({"question": QUESTION})

# 結果を表示
print(response)


旭化成のデジタルトランスフォーメーション（DX）への取り組みは、企業の競争力を高めるために重要な戦略の一環として進められています。以下に、これまでの取り組み、現状の問題、今後必要な取り組みについてまとめます。

### これまでのDXへの取り組み
1. **データ活用の推進**: 旭化成は、製造プロセスや製品開発においてデータ分析を活用し、効率化や品質向上を図ってきました。IoT技術を導入し、リアルタイムでのデータ収集と分析を行っています。

2. **業務プロセスのデジタル化**: 業務の効率化を目的に、ERPシステムやクラウドサービスの導入を進め、業務プロセスのデジタル化を推進しています。

3. **新技術の導入**: AIや機械学習を活用した新製品の開発や、製造工程の最適化に取り組んでいます。これにより、製品の競争力を高めることを目指しています。

### 現状の問題
1. **組織文化の変革**: DXを推進するためには、従来の業務慣行や組織文化を変える必要がありますが、これが難航している場合があります。特に、デジタル技術に対する理解や受け入れが不十分な部門が存在することが課題です。

2. **データのサイロ化**: 各部門でデータが分散して管理されているため、全社的なデータ活用が進まないという問題があります。データの統合と共有が求められています。

3. **人材不足**: DXを推進するための専門知識を持った人材が不足していることも課題です。特に、データサイエンティストやITエンジニアの確保が難しい状況です。

### 今後必要な取り組み
1. **組織文化の醸成**: DXを推進するための組織文化を育成するために、教育や研修を強化し、全社員がデジタル技術を理解し活用できる環境を整える必要があります。

2. **データ統合プラットフォームの構築**: 各部門のデータを統合し、全社的に活用できるデータプラットフォームを構築することが重要です。これにより、データの可視化や分析が容易になり、意思決定の質が向上します。

3. **人材育成と確保**: DXを推進するための人材を育成し、外部からの採用も含めて確保する取り組みが必要です。特に、データ分析やAI技術に精通した人材の育成が急務です。

4. **オープンイノベーションの推進**: 外部の企業や研

## 工夫なしのRAGを使う場合

In [23]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke(QUESTION)

'文脈には旭化成の具体的なDXへの取り組みや現状の問題についての情報は含まれていませんが、一般的な製造業におけるDXの課題や必要な取り組みについての情報が示されています。\n\n### これまでのDXへの取り組み\n- 製造業全体でのDX投資の喚起策（税制や補助金など）を通じて、製造業のデジタル化を進めてきた。\n- 産学官の連携による製造業DXの指針や評価指標の作成が進められている。\n\n### 現状の問題\n- DX投資が進まないという課題があり、部分最適に留まっているケースが多い。\n- 経営課題や業務変革課題の特定が不十分で、全体最適を目指す取り組みが不足している。\n- デジタル化の取組が個別工程のカイゼンにとどまり、製造プロセス全体を俯瞰した最適化が進んでいない。\n\n### 今後必要な取り組み\n- 経営層による経営・業務変革課題の特定を起点とし、各部門が連携して業務プロセスの変革を実施することが求められる。\n- 製造業系サービス事業者の育成や、製造業DX技術を持つ業種への支援が必要。\n- データ連携の進展や標準化の推進、リスキリングを通じた人材育成が重要である。\n\n旭化成に特有の情報は提供されていないため、具体的な取り組みや問題については、旭化成の公式発表や関連資料を参照することをお勧めします。'



```
# これはコードとして書式設定されます
```

## HyDE（Hypothetical Document Embeddings）


In [24]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。

質問: {question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

In [25]:
hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke(QUESTION)

'旭化成のこれまでのDXへの取り組みは、全員参加、現場主導、共創を基本にしたデジタルノーマルの実現を目指しています。具体的には、全従業員を「デジタル活用人財」として育成し、高度なデジタル技術やデータを活用して事業課題の解決やビジネスモデルの創出を進めています。特に「旭化成 DX Open Badgeプログラム」を通じて、生成AIコースやデジタルプロフェッショナル人財育成コースを新設し、コミュニティ活動を活性化させています。また、2019年度からは「パワーユーザー育成」プログラムを実施し、データ分析を牽引できる人財の育成に注力しています。\n\n現状の問題としては、デジタル化の取り組みが部分最適に留まっていることが挙げられます。製造プロセス全体を視野に入れた最適化が進んでいないため、経営課題や業務変革課題の根本的な解決には繋がりにくい状況です。また、各部門機能を総合的に捉えられる人材の不足や、進め方のノウハウの不足が変革のボトルネックとなっています。\n\n今後必要な取り組みとしては、製造プロセス全体を視野に入れた全体最適の推進が求められます。具体的には、デジタルツールを活用してリアルタイムでのデータ一元管理を行い、各部門間の情報連携を強化することが重要です。また、データ分析やデジタル技術に精通した人材の育成をさらに進め、業務変革を実現するためのノウハウを蓄積することが必要です。'

## 複数の検索クエリの生成


In [26]:
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリスト")


query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文書を検索するために、
3つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標です。

質問: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

In [27]:
multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke(QUESTION)

'### 旭化成のこれまでのDXへの取り組み\n1. **デジタル人財育成プログラム**: 旭化成は「旭化成DX Open Badgeプログラム」を通じて、全従業員を対象にデジタルスキルの向上を図ってきました。特に、生成AIコースやデジタルプロフェッショナル人財育成コースを新設し、デジタル技術の活用を促進しています。\n\n2. **マテリアルズ・インフォマティクス（MI）の活用**: MIを用いた開発が進められ、特にウイルス除去フィルター「プラノバ™」の高性能化に成功しました。これにより、従来の方法に比べて実験にかかる時間を大幅に削減し、効率的な開発が実現されています。\n\n3. **全員参加・現場主導のアプローチ**: DXを現場主導で進めるため、全従業員を「デジタル活用人財」として育成し、デジタル技術を活用した業務変革を推進しています。\n\n### 現状の問題\n1. **DX人材不足**: DXの推進において、専門的なデジタルスキルを持つ人材が不足していることが大きな課題となっています。\n\n2. **部分最適の傾向**: DXが進む中で、経営課題や業務変革課題の特定が不十分なまま進められることが多く、全体最適ではなく部分最適に留まるケースが見受けられます。\n\n3. **データ活用の限界**: 既存の業務プロセスにおけるデータの活用が十分でなく、デジタル技術を最大限に活用できていない状況があります。\n\n### 今後必要な取り組み\n1. **DX人材のさらなる育成**: デジタルスキルを持つ人材を増やすため、リスキリングや新たな教育プログラムの導入が必要です。特に、製造業に特化したIT人材の育成が求められます。\n\n2. **全体最適の推進**: 経営層が業務変革課題を特定し、各部門が連携して業務プロセスの変革を実施することで、全体最適を目指す必要があります。\n\n3. **データ連携の強化**: 企業間のデータ連携を進め、共通基盤の整備を行うことで、より効果的なデータ活用を実現することが重要です。\n\n4. **デジタル技術の標準化**: 製造業におけるデジタル技術の標準化を進め、業務プロセスの効率化を図ることが求められます。'

## RAG Fusion


In [28]:
from langchain_core.documents import Document


def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    # 各ドキュメントのコンテンツ (文字列) とそのスコアの対応を保持する辞書を準備
    content_score_mapping = {}

    # 検索クエリごとにループ
    for docs in retriever_outputs:
        # 検索結果のドキュメントごとにループ
        for rank, doc in enumerate(docs):
            content = doc.page_content

            # 初めて登場したコンテンツの場合はスコアを0で初期化
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1 / (順位 + k)) のスコアを加算
            content_score_mapping[content] += 1 / (rank + k)

    # スコアの大きい順にソート
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)  # noqa
    return [content for content, _ in ranked]

In [29]:
rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke(QUESTION)

'旭化成のこれまでのDXへの取り組みは、主に以下のような内容が挙げられます。\n\n### これまでのDXへの取り組み\n1. **マテリアルズ・インフォマティクス（MI）の活用**: MIを用いて、ウイルス除去フィルター「プラノバ™」の高性能化を実現し、従来の方法に比べて実験にかかる時間を大幅に削減しました。\n2. **デジタル人財育成**: 「旭化成DX Open Badgeプログラム」を通じて、全従業員をデジタル活用人財へ育成し、特にパワーユーザー育成プログラムを実施して、データ分析能力を高めています。\n3. **デジタル技術の導入**: スマートファクトリーやマーケティングオートメーションなど、さまざまなデジタルツールを導入し、業務の効率化を図っています。\n\n### 現状の問題\n1. **DX人材不足**: DX推進に必要な人材が不足しており、全体最適が進まない要因となっています。\n2. **部分最適の傾向**: 経営課題や業務変革課題の特定が不十分なままDXが進められると、既存業務の最適化にとどまり、全体最適が実現できないケースが多いです。\n3. **デジタル技術の活用の限界**: デジタル技術を活用した改革が現場主導で継続的に行われていないため、効果的な変革が進まない状況です。\n\n### 今後必要な取り組み\n1. **全体最適の推進**: DX人材の育成を強化し、経営層と業務部門が連携して業務プロセスの変革を進める必要があります。\n2. **デジタル技術の標準化と共通基盤の整備**: 企業間のデータ連携を進め、製造業DXの指針や評価指標を策定することが求められます。\n3. **リスキリングと人材の呼び込み**: 製造業現場を熟知するIT人材の育成や、製造業の魅力を向上させる取り組みが必要です。\n\nこれらの取り組みを通じて、旭化成はDXを加速し、競争力を高めることが期待されます。'

### Cohere のリランクモデルを使用する準備


In [30]:
os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")

SecretNotFoundError: Secret COHERE_API_KEY does not exist.

In [ ]:
!pip install langchain-cohere==0.3.0

### Cohere のリランクモデルの導入


In [ ]:
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document


def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
    question = inp["question"]
    documents = inp["documents"]

    cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)
    return cohere_reranker.compress_documents(documents=documents, query=question)


rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriever,
    }
    | RunnablePassthrough.assign(context=rerank)
    | prompt | model | StrOutputParser()
)

rerank_rag_chain.invoke("LangChainの概要を教えて")

## 6.5. 複数の Retriever を使う工夫


### LLM によるルーティング


In [31]:
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

In [32]:
from enum import Enum


class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"


class RouteOutput(BaseModel):
    route: Route


route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。

質問: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)

In [35]:
from typing import Any

def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)

    raise ValueError(f"Unknown route: {route}")


route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

In [36]:
route_rag_chain.invoke("旭化成の次の目標は？")

'旭化成の次の目標は、2030年度近傍において営業利益4,000億円、ROE15%以上、ROIC10%以上の達成を目指すことです。また、2030年度に2013年度比30%以上のGHG（温室効果ガス）排出量削減も目指しています。'

In [38]:
route_rag_chain.invoke("延岡市の2/19の天気は？")

'延岡市の2月19日（水）の天気は、最高気温11°C、最低気温0°Cで、降水確率は1%です。'

### ハイブリッド検索の実装


In [39]:
!pip install rank-bm25==0.2.2

In [40]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config(
    {"run_name": "chroma_retriever"}
)

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)

In [41]:
from langchain_core.runnables import RunnableParallel

hybrid_retriever = (
    RunnableParallel({
        "chroma_documents": chroma_retriever,
        "bm25_documents": bm25_retriever,
    })
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion
)

In [42]:
hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt | model | StrOutputParser()
)

hybrid_rag_chain.invoke(QUESTION)

'文脈には旭化成に関する具体的な情報は含まれていませんが、一般的な製造業のDX（デジタルトランスフォーメーション）に関する取り組みや課題についての情報が提供されています。以下は、製造業全般におけるDXの取り組み、現状の問題、今後必要な取り組みについての一般的な考察です。\n\n### これまでのDXへの取り組み\n1. **製造業DXの指針の策定**: 製造業者は、DXを進めるための指針や評価指標を作成し、戦略的な投資を行ってきた。\n2. **デジタル技術の導入**: IoTやデータ分析技術を活用し、設備の稼働率の見える化や業務プロセスの最適化を図っている。\n3. **人材育成**: DX推進のための人材育成やリスキリングに取り組んでいる。\n\n### 現状の問題\n1. **部分最適の傾向**: 多くの企業が既存の業務や部門の範囲内での最適化に留まっており、全体最適が実現できていない。\n2. **人材不足**: 製造プロセス全体を俯瞰できる人材が不足しており、業務変革のボトルネックとなっている。\n3. **ROIの不明確さ**: DX投資に対するリターンが明確でないため、経営層の理解や支持が得られにくい。\n\n### 今後必要な取り組み\n1. **全体最適の推進**: 部門を超えた業務プロセスの最適化を目指し、経営層と各部門が連携して取り組む必要がある。\n2. **データ連携の強化**: 企業間でのデータ連携を進め、SCM（サプライチェーンマネジメント）の最適化を図る。\n3. **標準化の推進**: 製造ソリューションやデータ連携の手法についての標準化を進め、業界全体の競争力を高める。\n4. **人材育成の強化**: DX推進に必要なスキルを持つ人材の育成を強化し、製造業の魅力を向上させることでテック系人材を呼び込む。\n\n旭化成に特有の情報が必要な場合は、具体的なデータや事例を参照することが望ましいです。'